#Model ensembling/Data augmentation TP (Nicolas & Richard)

* *Loading the Data*

In [2]:
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 451 kB 4.8 MB/s 
     |████████████████████████████████| 212 kB 89.0 MB/s 
     |████████████████████████████████| 115 kB 88.6 MB/s 
     |████████████████████████████████| 182 kB 69.5 MB/s 
     |████████████████████████████████| 127 kB 87.1 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [3]:
# https://huggingface.co/datasets/allocine

from datasets import load_dataset_builder
ds_builder = load_dataset_builder("allocine")

# Inspect dataset description
print(ds_builder.info.description)

# Inspect dataset features
print(ds_builder.info.features)

# get_dataset_split_names
from datasets import get_dataset_split_names
get_dataset_split_names("allocine")

# load_dataset
from datasets import load_dataset
allocine_dataset = load_dataset("allocine")
allocine_dataset
#train_dataset = load_dataset("allocine", split="train")

 Allocine Dataset: A Large-Scale French Movie Reviews Dataset.
 This is a dataset for binary sentiment classification, made of user reviews scraped from Allocine.fr.
 It contains 100k positive and 100k negative reviews divided into 3 balanced splits: train (160k reviews), val (20k) and test (20k).

{'review': Value(dtype='string', id=None), 'label': ClassLabel(names=['neg', 'pos'], id=None)}


Generating train split:   0%|          | 0/160000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/20000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/20000 [00:00<?, ? examples/s]

Dataset allocine downloaded and prepared to /root/.cache/huggingface/datasets/allocine/allocine/1.0.0/ea86b1dc05eae3a45a07b6281f2d4033b5fe7927b1008d06aa457ca1eae660d0. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['review', 'label'],
        num_rows: 160000
    })
    validation: Dataset({
        features: ['review', 'label'],
        num_rows: 20000
    })
    test: Dataset({
        features: ['review', 'label'],
        num_rows: 20000
    })
})

In [4]:
import numpy as np

def get_dataset_split_ratio(dataset_split, seed=42, ratio=100):
  if ratio > 100 or ratio < 1: ratio = 100
  reviews = dataset_split['review']
  labels = dataset_split['label']
  
  data = [pair for pair in zip(reviews, labels)]
  
  np.random.seed(seed)
  np.random.shuffle(data)
  
  ratio_len = int(len(data)*ratio/100)
  
  data = list(zip(*data))

  return {'review':data[0][:ratio_len], 'label':data[1][:ratio_len]}


In [5]:
import pandas as pd

# From huggingface dataset split to DataFrame
def hf_dataset_split_to_df (huggingface_dataset_split):
  df = pd.DataFrame(huggingface_dataset_split)
  # https://github.com/amaiya/ktrain/blob/master/examples/text/ArabicHotelReviews-nbsvm.ipynb
  df['label'] = df['label'].apply(lambda x: 'negative' if x == 0 else 'positive')
  df = pd.concat([df, df.label.astype('str').str.get_dummies()], axis=1, sort=False)
  df = df[['review', 'negative', 'positive']]
  return df

In [6]:
ratio = 50 # percent of the data

train_df = hf_dataset_split_to_df(get_dataset_split_ratio(allocine_dataset['train'], 42, ratio))
val_df = hf_dataset_split_to_df(get_dataset_split_ratio(allocine_dataset['validation'], 42, ratio))
test_df = hf_dataset_split_to_df(get_dataset_split_ratio(allocine_dataset['test'], 42, ratio))

print(train_df.head())
print(train_df.describe())
print('len(train_df) :', len(train_df))
print('len(val_df) :', len(val_df))
print('len(test_df) :', len(test_df))

                                              review  negative  positive
0  Un excellent thriller d'action où les scènes d...         0         1
1  Si le scénariste, qui aurait pu faire un minim...         1         0
2  Référence dans la filmographie de Bogart, "Le ...         1         0
3  Un bon scénario, un bon film, une histoire lou...         0         1
4  Un scenario vide et une mise en scene trés sop...         1         0
           negative      positive
count  80000.000000  80000.000000
mean       0.495000      0.505000
std        0.499978      0.499978
min        0.000000      0.000000
25%        0.000000      0.000000
50%        0.000000      1.000000
75%        1.000000      1.000000
max        1.000000      1.000000
len(train_df) : 80000
len(val_df) : 10000
len(test_df) : 10000


#Ktrain configuration

In [7]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID";
os.environ["CUDA_VISIBLE_DEVICES"]="0"; 


In [8]:
!pip install ktrain

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 25.3 MB 4.7 MB/s 
     |████████████████████████████████| 981 kB 59.3 MB/s 
     |████████████████████████████████| 263 kB 82.4 MB/s 
     |████████████████████████████████| 3.8 MB 82.0 MB/s 
     |████████████████████████████████| 1.3 MB 49.6 MB/s 
     |████████████████████████████████| 468 kB 50.4 MB/s 
     |████████████████████████████████| 7.6 MB 77.5 MB/s 
     |████████████████████████████████| 880 kB 86.0 MB/s 
  Created wheel for ktrain: filename=ktrain-0.31.10-py3-none-any.whl size=25312982 sha256=c339d6756ba64bbe61771051b88101f3d104e627ff2988583c695b77080e4f1f
  Stored in directory: /root/.cache/pip/wheels/9c/1c/1b/6df2db85720b8f5c6ea5e3ae37313cfc656f248abf910b7cfd
  Created wheel for keras-bert: filename=keras_bert-0.89.0-py3-none-any.whl size=33516 sha256=40bbfdcc9583e6a24463bb694e5679c72dd2e5ebc8709a25958de70e298ee732
  Stored in dire

In [9]:
# Execution time 30 s
import ktrain
from ktrain import text

*Preprocessing the data*

In [10]:

(x_train_preproc, y_train_preproc), (x_val_preproc, y_val_preproc), preproc = text.texts_from_df (train_df, 
                                                                   'review', # name of column containing review text
                                                                   label_columns=['negative', 'positive'],
                                                                   val_df=val_df, # if None, 10% of data will be used for validation
                                          #max_features=NUM_WORDS, 
                                          #maxlen=MAXLEN,
                                          #ngram_range=NGRAMS_SIZE,
                                          preprocess_mode='standard' # default
                                          )



['negative', 'positive']
   negative  positive
0         0         1
1         1         0
2         1         0
3         0         1
4         1         0
['negative', 'positive']
   negative  positive
0         0         1
1         0         1
2         1         0
3         1         0
4         1         0
language: fr
Word Counts: 135007
Nrows: 80000
80000 train sequences
train sequence lengths:
	mean : 87
	95percentile : 240
	99percentile : 303
x_train shape: (80000,400)
y_train shape: (80000, 2)
Is Multi-Label? False
10000 test sequences
test sequence lengths:
	mean : 88
	95percentile : 246
	99percentile : 306
x_test shape: (10000,400)
y_test shape: (10000, 2)


#FASTTEXT MODEL

In [11]:
# Build and return a text classification model https://amaiya.github.io/ktrain/text/index.html#ktrain.text.text_classifier
fasttext_model = text.text_classifier('fasttext', (x_train_preproc, y_train_preproc), preproc=preproc)

# Returns a Learner instance that can be used to tune and train Keras models https://amaiya.github.io/ktrain/index.html#ktrain.get_learner
fasttext_learner = ktrain.get_learner(fasttext_model, train_data=(x_train_preproc, y_train_preproc), val_data=(x_val_preproc, y_val_preproc))

Is Multi-Label? False
compiling word ID features...
maxlen is 400
done.


In [12]:
LEARNING_RATE = 0.01

fasttext_learner.autofit(LEARNING_RATE)

early_stopping automatically enabled at patience=5
reduce_on_plateau automatically enabled at patience=2


begin training using triangular learning rate policy with max lr of 0.01...
Epoch 1/1024
2500/2500 [==============================] - 16s 5ms/step - loss: 0.4116 - accuracy: 0.8114 - val_loss: 0.2512 - val_accuracy: 0.9027
Epoch 2/1024
2500/2500 [==============================] - 11s 5ms/step - loss: 0.2977 - accuracy: 0.8784 - val_loss: 0.2321 - val_accuracy: 0.9117
Epoch 3/1024
2500/2500 [==============================] - 11s 5ms/step - loss: 0.2762 - accuracy: 0.8894 - val_loss: 0.2253 - val_accuracy: 0.9131
Epoch 4/1024
2500/2500 [==============================] - 11s 5ms/step - loss: 0.2547 - accuracy: 0.8991 - val_loss: 0.2203 - val_accuracy: 0.9127
Epoch 5/1024
2500/2500 [==============================] - 11s 5ms/step - loss: 0.2437 - accuracy: 0.9035 - val_loss: 0.2245 - val_accuracy: 0.9140
Epoch 6/1024
2489/2500 [============================>.] - ETA: 0s - loss: 0.2347 -

In [13]:
# save Predictor (i.e., model and Preprocessor instance) after partially training
ktrain.get_predictor(fasttext_learner.model, preproc).save('fasttext_allocine.model+preproc')
fasttext_predictor = ktrain.get_predictor(fasttext_learner.model, preproc)

*Evaluation*

In [14]:
# reviews 
x_test = list(test_df['review'])

# labels (gold) 
y_test = list(test_df['positive'])

In [15]:
# hypothèse
y_hyp = [0 if h == 'negative' else 1 for h in fasttext_predictor.predict(x_test) ]
y_hyp_fastext= [0 if h == 'negative' else 1 for h in fasttext_predictor.predict(x_test) ]#we are going to use this variable later on for the ensembling method

313/313 [==============================] - 0s 1ms/step


In [16]:
from sklearn.metrics import f1_score, precision_score, recall_score

f1_positive_fasttext = f1_score(y_test, y_hyp)

In [17]:
f1_score(y_test, y_hyp, average = "binary", pos_label = 1)

0.9159420289855073

#NBSVM

In [18]:
# load an NBSVM model
nbsvm_model = text.text_classifier('nbsvm', (x_train_preproc, y_train_preproc), preproc=preproc)
nbsvm_learner = ktrain.get_learner(nbsvm_model, train_data=(x_train_preproc, y_train_preproc), val_data=(x_val_preproc, y_val_preproc))

# fine tune
LEARNING_RATE = 0.01
nbsvm_learner.autofit(LEARNING_RATE)



Is Multi-Label? False
compiling word ID features...
maxlen is 400
building document-term matrix... this may take a few moments...
rows: 1-10000
rows: 10001-20000
rows: 20001-30000
rows: 30001-40000
rows: 40001-50000
rows: 50001-60000
rows: 60001-70000
rows: 70001-80000
computing log-count ratios...
done.
early_stopping automatically enabled at patience=5
reduce_on_plateau automatically enabled at patience=2


begin training using triangular learning rate policy with max lr of 0.01...
Epoch 1/1024
2500/2500 [==============================] - 11s 4ms/step - loss: 0.2516 - accuracy: 0.9107 - val_loss: 0.2249 - val_accuracy: 0.9185
Epoch 2/1024
2500/2500 [==============================] - 10s 4ms/step - loss: 0.1717 - accuracy: 0.9367 - val_loss: 0.2497 - val_accuracy: 0.9127
Epoch 3/1024
2499/2500 [============================>.] - ETA: 0s - loss: 0.1495 - accuracy: 0.9452
Epoch 00003: Reducing Max LR on Plateau: new max lr will be 0.005 (if not early_stopping).
2500/2500 [===============

In [19]:
# données
x_test = list(test_df['review']) # reviews 
y_test = list(test_df['positive']) # labels (gold) 

# prédiction
nbsvm_predictor = ktrain.get_predictor(nbsvm_learner.model, preproc)
y_hyp = [0 if h == 'negative' else 1 for h in nbsvm_predictor.predict(x_test) ]
y_hyp_NBSVM = [0 if h == 'negative' else 1 for h in nbsvm_predictor.predict(x_test) ]

f1_score_nbsvm=f1_score(y_test, y_hyp)
# évaluation
print(f1_score(y_test, y_hyp))



313/313 [==============================] - 0s 1ms/step
0.9195568899471995


In [20]:
# save Predictor (i.e., model and Preprocessor instance) after partially training
ktrain.get_predictor(nbsvm_learner.model, preproc).save('nbsvm_allocine.model+preproc')


#BERT

In [21]:
# ETAPE 1 
(x_train_preproc, y_train_preproc), (x_val_preproc, y_val_preproc), preproc = text.texts_from_df (train_df, 
                      'review',
                      label_columns = ["negative", "positive"],
                      val_df= val_df, # if None, 10% of data will be used for validation
                      ##max_features=NUM_WORDS, 
                      #maxlen=MAXLEN,
                      preprocess_mode='bert' 
                      )

['negative', 'positive']
   negative  positive
0         0         1
1         1         0
2         1         0
3         0         1
4         1         0
['negative', 'positive']
   negative  positive
0         0         1
1         0         1
2         1         0
3         1         0
4         1         0
downloading pretrained BERT model (multi_cased_L-12_H-768_A-12.zip)...
[██████████████████████████████████████████████████]
extracting pretrained BERT model...
done.

cleanup downloaded zip...
done.

preprocessing train...
language: fr


Is Multi-Label? False
preprocessing test...
language: fr


In [22]:
# ETAPE 2 et 3
bert_model = text.text_classifier('bert', (x_train_preproc, y_train_preproc) , preproc=preproc)
bert_learner = ktrain.get_learner(bert_model, 
                             train_data=(x_train_preproc, y_train_preproc), 
                             val_data=(x_val_preproc, y_val_preproc), 
                             batch_size=6)
# ETAPE 5
bert_learner.fit_onecycle(2e-5, 1)

Is Multi-Label? False
maxlen is 400
done.


begin training using onecycle policy with max lr of 2e-05...
13334/13334 [==============================] - 1427s 105ms/step - loss: 0.2188 - accuracy: 0.9097 - val_loss: 0.1317 - val_accuracy: 0.9502


*Evaluation*

In [23]:
# données
x_test = list(test_df['review'])   # reviews 
y_test = list(test_df['positive']) # labels (gold) 

# prédiction
bert_predictor = ktrain.get_predictor(bert_learner.model, preproc)
y_hyp = [0 if h == 'negative' else 1 for h in bert_predictor.predict(x_test) ]
y_hyp_bert = [0 if h == 'negative' else 1 for h in bert_predictor.predict(x_test) ]

# évaluation
f1_score_bert=f1_score(y_test, y_hyp)
from sklearn.metrics import f1_score, precision_score, recall_score
print(f1_score(y_test, y_hyp))

313/313 [==============================] - 37s 119ms/step
0.9518248175182482


In [24]:
# save Predictor (i.e., model and Preprocessor instance) after partially training
ktrain.get_predictor(bert_learner.model, preproc).save('bert_allocine.model+preproc')

# reload Predictor and extract model
#model = ktrain.load_predictor('/tmp/my_predictor').model

#LOGREG

In [25]:

(x_train_preproc, y_train_preproc), (x_val_preproc, y_val_preproc), preproc = text.texts_from_df (train_df, 
                                                                   'review', # name of column containing review text
                                                                   label_columns=['negative', 'positive'],
                                                                   val_df=val_df, # if None, 10% of data will be used for validation
                                          #max_features=NUM_WORDS, 
                                          #maxlen=MAXLEN,
                                          #ngram_range=NGRAMS_SIZE,
                                          preprocess_mode='standard' # default
                                          )



['negative', 'positive']
   negative  positive
0         0         1
1         1         0
2         1         0
3         0         1
4         1         0
['negative', 'positive']
   negative  positive
0         0         1
1         0         1
2         1         0
3         1         0
4         1         0
language: fr
Word Counts: 135007
Nrows: 80000
80000 train sequences
train sequence lengths:
	mean : 87
	95percentile : 240
	99percentile : 303
x_train shape: (80000,400)
y_train shape: (80000, 2)
Is Multi-Label? False
10000 test sequences
test sequence lengths:
	mean : 88
	95percentile : 246
	99percentile : 306
x_test shape: (10000,400)
y_test shape: (10000, 2)


In [26]:
# Build and return a text classification model https://amaiya.github.io/ktrain/text/index.html#ktrain.text.text_classifier
logreg_model = text.text_classifier('logreg', (x_train_preproc, y_train_preproc), preproc=preproc)

# Returns a Learner instance that can be used to tune and train Keras models https://amaiya.github.io/ktrain/index.html#ktrain.get_learner
logreg_learner = ktrain.get_learner(logreg_model, train_data=(x_train_preproc, y_train_preproc), val_data=(x_val_preproc, y_val_preproc))

Is Multi-Label? False
compiling word ID features...
maxlen is 400
done.


In [27]:
LEARNING_RATE = 0.01
logreg_learner.autofit(LEARNING_RATE)

early_stopping automatically enabled at patience=5
reduce_on_plateau automatically enabled at patience=2


begin training using triangular learning rate policy with max lr of 0.01...
Epoch 1/1024
2500/2500 [==============================] - 11s 4ms/step - loss: 0.2998 - accuracy: 0.8929 - val_loss: 0.2602 - val_accuracy: 0.9095
Epoch 2/1024
2500/2500 [==============================] - 10s 4ms/step - loss: 0.1973 - accuracy: 0.9305 - val_loss: 0.2963 - val_accuracy: 0.9092
Epoch 3/1024
2496/2500 [============================>.] - ETA: 0s - loss: 0.1743 - accuracy: 0.9387
Epoch 00003: Reducing Max LR on Plateau: new max lr will be 0.005 (if not early_stopping).
2500/2500 [==============================] - 10s 4ms/step - loss: 0.1742 - accuracy: 0.9387 - val_loss: 0.3332 - val_accuracy: 0.9069
Epoch 4/1024
2500/2500 [==============================] - 10s 4ms/step - loss: 0.1245 - accuracy: 0.9549 - val_loss: 0.3308 - val_accuracy: 0.9088
Epoch 5/1024
2487/2500 [===========================

In [28]:
# données
x_test = list(test_df['review']) # reviews 
y_test = list(test_df['positive']) # labels (gold) 

# prédiction
logreg_predictor = ktrain.get_predictor(logreg_learner.model, preproc)
y_hyp = [0 if h == 'negative' else 1 for h in logreg_predictor.predict(x_test) ]


f1_score_logreg=f1_score(y_test, y_hyp)
# évaluation
print(f1_score(y_test, y_hyp))


313/313 [==============================] - 0s 1ms/step
0.9106197446278417


#CAMEMBERT

In [29]:
x_train = list(train_df['review'])
y_train = list(train_df['positive'])

x_val = list(val_df['review'])
y_val = list(val_df['positive'])

x_test = list(test_df['review'])
y_test = list(test_df['positive'])

In [30]:
import ktrain
from ktrain import text
MODEL_NAME = 'camembert-base'
#MODEL_NAME = 'albert-base-v2'
#MODEL_NAME = 'camembert-base'

CLASS_NAMES = ["negative", "positive"]

camembert_preproc = text.Transformer(MODEL_NAME, maxlen=500, class_names=CLASS_NAMES)
train_preproc = camembert_preproc.preprocess_train(x_train, y_train)
val_preproc = camembert_preproc.preprocess_test(x_val, y_val)
 
camembert_model = camembert_preproc.get_classifier()
camembert_learner = ktrain.get_learner(camembert_model, train_data=train_preproc, val_data=val_preproc, batch_size=12)

Downloading:   0%|          | 0.00/508 [00:00<?, ?B/s]

preprocessing train...
language: fr
train sequence lengths:
	mean : 91
	95percentile : 253
	99percentile : 320


Downloading:   0%|          | 0.00/792k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.33M [00:00<?, ?B/s]

Is Multi-Label? False
preprocessing test...
language: fr
test sequence lengths:
	mean : 93
	95percentile : 260
	99percentile : 323


Downloading:   0%|          | 0.00/518M [00:00<?, ?B/s]

In [31]:
camembert_learner.fit_onecycle(0.01, 1)




begin training using onecycle policy with max lr of 0.01...
6667/6667 [==============================] - 1308s 191ms/step - loss: 0.7102 - accuracy: 0.4998 - val_loss: 0.6933 - val_accuracy: 0.4942


In [32]:
# données
x_test = list(test_df['review'])   # reviews 
y_test = list(test_df['positive']) # labels (gold) 

# prédiction
camembert_predictor = ktrain.get_predictor(camembert_learner.model, camembert_preproc)
y_hyp = [0 if h == 'negative' else 1 for h in camembert_predictor.predict(x_test) ]


f1_score_camembert=f1_score(y_test, y_hyp)
# évaluation
print(f1_score(y_test, y_hyp))


313/313 [==============================] - 48s 146ms/step
0.64810058131675


#MODEL ENSEMBLING BY SAMPLE AVERAGING METHOD

In [33]:
f1_score_averaged = (f1_score_bert + f1_score_logreg + f1_score_nbsvm) / 3

print(f1_score_averaged)

0.9273338173644298


#MODEL ENSEMBLING BY STACKING

*J'AI TROUVÉ LE CODE POUR CETTE PARTIE "BYSTACKING" MAIS JE L'AI PAS PARFAITEMENT COMPRIS ET DONC JE L'AI PAS RAJOUTÉ*

#MODEL ENSEMBLING BY HARD VOTING

In [34]:
def get_hard_vote(predictors, data):
  predictions = []
  for predictor in predictors:
    predictions.append(predictor.predict(data)[0])
  return max(set(predictions), key=predictions.count)

In [36]:
predictors = [fasttext_predictor, nbsvm_predictor, bert_predictor]

In [37]:


test_df_eval = hf_dataset_split_to_df(get_dataset_split_ratio(allocine_dataset['test'], 42, 5))

x_test = list(test_df_eval['review'])  
y_test = list(test_df_eval['positive']) 

hard_predictions = []
for data in x_test:
  hard_predictions.append(get_hard_vote(predictors, data))

y_hyp_hardvoting = [0 if h == 'negative' else 1 for h in hard_predictions]

print(f1_score(y_test, y_hyp_hardvoting))

1/1 [==============================] - 0s 34ms/step
0.6577181208053691


#DATA AUGMENTATION

In [ ]:
!pip3 install textattack[tensorflow]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 418 kB 6.7 MB/s 
     |████████████████████████████████| 5.5 MB 80.3 MB/s 
     |████████████████████████████████| 41.4 MB 1.3 MB/s 
     |████████████████████████████████| 401 kB 75.4 MB/s 
     |████████████████████████████████| 365 kB 76.6 MB/s 
     |████████████████████████████████| 60 kB 10.3 MB/s 
     |████████████████████████████████| 769 kB 76.4 MB/s 
     |████████████████████████████████| 125 kB 68.0 MB/s 
     |████████████████████████████████| 3.6 MB 69.9 MB/s 
     |████████████████████████████████| 5.8 MB 32.5 MB/s 
     |████████████████████████████████| 511.7 MB 4.9 kB/s 
     |████████████████████████████████| 125 kB 53.3 MB/s 
     |████████████████████████████████| 95 kB 6.5 MB/s 
     |████████████████████████████████| 5.9 MB 24.6 MB/s 
     |████████████████████████████████| 4.6 MB 77.1 MB/s 
     |████████████████████████████

In [ ]:
from textattack.attack_recipes import PWWSRen2019
from textattack.datasets import HuggingFaceDataset
from textattack.models.wrappers import ModelWrapper
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification, pipeline
from textattack import Attacker

import numpy as np

# Quiet TensorFlow.
import os
if "TF_CPP_MIN_LOG_LEVEL" not in os.environ:
    os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"


class HuggingFaceSentimentAnalysisPipelineWrapper(ModelWrapper):
    """ Transformers sentiment analysis pipeline returns a list of responses
        like

            [{'label': 'POSITIVE', 'score': 0.7817379832267761}]

        We need to convert that to a format TextAttack understands, like

            [[0.218262017, 0.7817379832267761]
    """
    def __init__(self, model):
        self.model = model#pipeline = pipeline
    def __call__(self, text_inputs):
        raw_outputs = self.model(text_inputs)
        outputs = []
        for output in raw_outputs:
            score = output['score']
            if output['label'] == 'POSITIVE':
                outputs.append([1-score, score])
            else:
                outputs.append([score, 1-score])
        return np.array(outputs)


textattack: Updating TextAttack package dependencies.
textattack: Downloading NLTK required packages.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package omw to /root/nltk_data...
[nltk_data] Downloading package universal_tagset to /root/nltk_data...
[nltk_data]   Unzipping taggers/universal_tagset.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
# Create the model: a French sentiment analysis model.
# see https://github.com/TheophileBlard/french-sentiment-analysis-with-bert
model = TFAutoModelForSequenceClassification.from_pretrained("camembert-base")
tokenizer = AutoTokenizer.from_pretrained("camembert-base")
pipeline = pipeline('sentiment-analysis', model=model, tokenizer=tokenizer)

model_wrapper = HuggingFaceSentimentAnalysisPipelineWrapper(pipeline)

# Create the recipe: PWWS uses a WordNet transformation.
recipe = PWWSRen2019.build(model_wrapper)
#
# WordNet defaults to english. Set the default language to French ('fra')
#
# See "Building a free French wordnet from multilingual resources",
# E. L. R. A. (ELRA) (ed.),
# Proceedings of the Sixth International Language Resources and Evaluation (LREC’08).
recipe.transformation.language = 'fra'


# dataset = HuggingFaceDataset('allocine', split='test[:10%]')

# # datalist = ["Ce film était horrible ! L'intrigue était ennuyeuse. Le jeu d'acteur était correct, cependant.",
# #          "Le film est vraiment nul. Je veux qu'on me rende mon argent.",
# #         "Quelle belle comédie romantique. 10/10 à revoir !"]


# attacker = Attacker(recipe, dataset)
# attack_output = attacker.attack_dataset()
# print(attack_output)


All model checkpoint layers were used when initializing TFCamembertForSequenceClassification.

Some layers of TFCamembertForSequenceClassification were not initialized from the model checkpoint at camembert-base and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
textattack: Unknown if model of class <class 'transformers.pipelines.text_classification.TextClassificationPipeline'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.


In [ ]:

# Create the model: a French sentiment analysis model.
# see https://github.com/TheophileBlard/french-sentiment-analysis-with-bert
model = TFAutoModelForSequenceClassification.from_pretrained("Jean-Baptiste/camembert-ner",)
tokenizer = AutoTokenizer.from_pretrained("Jean-Baptiste/camembert-ner")
pipeline = pipeline('ner', model=model, tokenizer=tokenizer, aggregation_strategy="simple")

model_wrapper = HuggingFaceSentimentAnalysisPipelineWrapper(pipeline)

# Create the recipe: PWWS uses a WordNet transformation.
recipe = PWWSRen2019.build(model_wrapper)
#
# WordNet defaults to english. Set the default language to French ('fra')
#
# See "Building a free French wordnet from multilingual resources",
# E. L. R. A. (ELRA) (ed.),
# Proceedings of the Sixth International Language Resources and Evaluation (LREC’08).
recipe.transformation.language = 'fra'



In [ ]:
dataset = HuggingFaceDataset('allocine', split='test')

attacker = Attacker(recipe, dataset)
attacker.attack_dataset()

  0%|          | 0/3 [00:00<?, ?it/s]

textattack: Loading datasets dataset allocine, split test.


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  weighted-saliency
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapWordNet
  (constraints): 
    (0): RepeatModification
    (1): StopwordModification
  (is_black_box):  True
) 




 10%|█         | 1/10 [00:00<00:02,  3.09it/s]
[Succeeded / Failed / Skipped / Total] 0 / 0 / 1 / 1:  10%|█         | 1/10 [00:00<00:02,  3.05it/s]

--------------------------------------------- Result 1 ---------------------------------------------
[[Negative (50%)]] --> [[[SKIPPED]]]

Magnifique épopée, une belle histoire, touchante avec des acteurs qui interprètent très bien leur rôles (Mel Gibson, Heath Ledger, Jason Isaacs...), le genre de film qui se savoure en famille! :)




...


[Succeeded / Failed / Skipped / Total] 0 / 3 / 5 / 8:  90%|█████████ | 9/10 [10:11<01:07, 67.99s/it]
[Succeeded / Failed / Skipped / Total] 0 / 3 / 6 / 9:  90%|█████████ | 9/10 [10:11<01:07, 67.99s/it]

--------------------------------------------- Result 9 ---------------------------------------------
[[Negative (51%)]] --> [[[SKIPPED]]]

Un très joli film, qui ressemble à un téléfilm mais qui a le mérite d'être émouvant et proche de ses personnages. Magimel est vraiment très bon et l'histoire est touchante





[Succeeded / Failed / Skipped / Total] 0 / 3 / 6 / 9: 100%|██████████| 10/10 [33:09<00:00, 198.93s/it]
[Succeeded / Failed / Skipped / Total] 0 / 4 / 6 / 10: 100%|██████████| 10/10 [33:09<00:00, 198.94s/it]

--------------------------------------------- Result 10 ---------------------------------------------
[[Negative (51%)]] --> [[[FAILED]]]

Mais comment certaines personnes ont pus lui mettre 5/5 et donc dire indirectement que c'est un chef-d'œuvre ??? Et comment a-t-il fait pour sortir au cinéma et non en DTV ??? C'est pas un film que l'on regarde dans une salle obscur ça, pour moi ça ressemble plus à un téléfilm que l'on visionne un dimanche pluvieux pour que les enfants arrête de nous casser les pieds ! Et puis, le scénario avec le chien que devient le meilleur ami du gosse, c'est du vu et revu (un cliché) ! L'acteur principal est quant à lui aussi agaçant que son personnage ! Les suites ont l'air aussi mauvaises que Buddy Star des Paniers étant donné que l'histoire est quasiment la même (pour moi ça c'est pas des suites, c'est plutôt une succession de petits reboots inutiles). Reste regardable pour les moins de 10 ans (et encore, même moi à 6 ans, je n'aurais pas aimé).



+--------


/usr/local/lib/python3.7/dist-packages/textattack/metrics/attack_metrics/words_perturbed.py:83: RuntimeWarning: Mean of empty slice.
  average_perc_words_perturbed = self.perturbed_word_percentages.mean()
/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


20000

#PARTIE 3 - TP RICHARD - NER DETECTION & CLASSIFICATION USING NER FEATURES

In [13]:
# Then we create a list of all the entities that can be retrieved in the text 
# using hugging face model for french NER (PER, ORG, LOC)

from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline

tokenizer = AutoTokenizer.from_pretrained("Jean-Baptiste/camembert-ner")
model_nlp = AutoModelForTokenClassification.from_pretrained("Jean-Baptiste/camembert-ner")

nlp = pipeline('ner', model=model_nlp, tokenizer=tokenizer, aggregation_strategy="simple")

Downloading:   0%|          | 0.00/269 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/892 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/811k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/210 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/440M [00:00<?, ?B/s]

In [13]:
df_train = pd.DataFrame(train_df, columns=['review', 'negative','positive','per','misc','loc','org'])
df_test = pd.DataFrame(test_df, columns=['review', 'negative','positive','per','misc','loc','org'])
df_val = pd.DataFrame(val_df, columns=['review', 'negative','positive','per','misc','loc','org'])

In [ ]:
df_train=df_train.fillna(0)
df_test=df_test.fillna(0)

In [14]:
results=[]
test_df1=train_df
for i in range(0,len(test_df1)):
  results.append(nlp(test_df1['review'][i]))

In [15]:
results_test=[]
test_df2=test_df
for i in range(0,len(test_df2)):
  results_test.append(nlp(test_df2['review'][i]))

In [13]:
for i in range(len(results)):
  for j in range(len(results[i])):
    if results[i][j]['entity_group']=='PER':
      df_train['per'][i]+=1
    if results[i][j]['entity_group']=='MISC':
      df_train['misc'][i]+=1
    if results[i][j]['entity_group']=='LOC':
      df_train['loc'][i]+=1
    if results[i][j]['entity_group']=='ORG':
      df_train['org'][i]+=1



/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: Sett

In [14]:

for i in range(len(results_test)):
  for j in range(len(results_test[i])):
    if results_test[i][j]['entity_group']=='PER':
      df_test['per'][i]+=1
    if results_test[i][j]['entity_group']=='MISC':
      df_test['misc'][i]+=1
    if results_test[i][j]['entity_group']=='LOC':
      df_test['loc'][i]+=1
    if results_test[i][j]['entity_group']=='ORG':
      df_test['org'][i]+=1


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/py

In [19]:
df_val

,review,negative,positive,per,misc,loc,org
0,"Un film extraordinaire par sa légèreté, sa poé...",0,1,0.0,0.0,0.0,0.0
1,"Le Dernier Exorcisme, s'il n'est pas le métrag...",0,1,0.0,1.0,0.0,0.0
2,"Attention, ce film est noté sur l'echelle du C...",1,0,1.0,2.0,0.0,0.0
3,"Tout simplement ennuyant , une perte de temps,...",1,0,0.0,0.0,0.0,0.0
4,Très difficile de regarder un film d'horreur a...,1,0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...
1995,Un peu un fourre-tout de ce qui s'est déjà fai...,1,0,0.0,4.0,0.0,0.0
1996,Ce film a le mérite de nous assurer que Serena...,1,0,3.0,3.0,0.0,0.0
1997,Un classique du film social britannique la gra...,0,1,1.0,0.0,0.0,0.0
1998,Un excellent remake à la sauce coréenne. L'his...,0,1,1.0,0.0,0.0,0.0


In [12]:
df_train=df_train.fillna(0)
df_test=df_test.fillna(0)
df_val=df_val.fillna(0)

In [ ]:

(x_train_preproc, y_train_preproc), (x_val_preproc, y_val_preproc), preproc = text.texts_from_df (train_df, 
                                                                   ['review'], # name of column containing review text
                                                                   label_columns=['negative', 'positive'],
                                                                   val_df=val_df, # if None, 10% of data will be used for validation
                                          #max_features=NUM_WORDS, 
                                          #maxlen=MAXLEN,
                                          #ngram_range=NGRAMS_SIZE,
                                          preprocess_mode='standard' # default
                                          )



In [ ]:
lll=df_train.get(['per','misc'])
lll_test=df_test.get(['per','misc'])

ss=np.hstack([x_train_preproc,lll])
ss_2=np.hstack([x_val_preproc,lll_val])


In [65]:
# load an NBSVM model
nbsvm_model = text.text_classifier('nbsvm', (ss.astype(int), y_train_preproc), preproc=preproc)
nbsvm_learner = ktrain.get_learner(nbsvm_model, train_data=(ss, y_train_preproc), val_data=(ss_2.astype(int), y_val_preproc))

# fine tune
LEARNING_RATE = 0.01
nbsvm_learner.autofit(LEARNING_RATE)



Is Multi-Label? False
compiling word ID features...
maxlen is 402
building document-term matrix... this may take a few moments...
rows: 1-10000
rows: 10001-16000
computing log-count ratios...
done.
early_stopping automatically enabled at patience=5
reduce_on_plateau automatically enabled at patience=2


begin training using triangular learning rate policy with max lr of 0.01...
Epoch 1/1024
500/500 [==============================] - 3s 4ms/step - loss: 0.2941 - accuracy: 0.9029 - val_loss: 0.2537 - val_accuracy: 0.9115
Epoch 2/1024
500/500 [==============================] - 2s 4ms/step - loss: 0.1428 - accuracy: 0.9544 - val_loss: 0.2542 - val_accuracy: 0.9135
Epoch 3/1024
494/500 [============================>.] - ETA: 0s - loss: 0.1036 - accuracy: 0.9705
Epoch 00003: Reducing Max LR on Plateau: new max lr will be 0.005 (if not early_stopping).
500/500 [==============================] - 2s 4ms/step - loss: 0.1034 - accuracy: 0.9705 - val_loss: 0.2629 - val_accuracy: 0.9110
Epoch 4/102

In [8]:
!pip3 install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.5 MB 32.2 MB/s 
     |████████████████████████████████| 7.6 MB 78.0 MB/s 
